## Fundamentos de ETL com Python

- Extração
    - os dados são exraindos de diferentes fontes de dados
- Transformação
    - Onde são transformados e limpos
- Load
    - Carregar os dados no data warehouse

Transformação de Dados

In [2]:
import pandas as pd
import pandera as pa

In [3]:
# abrindo csv usando o separador ";", fazendo parse das datas e fazendo a limpeza dos dados, alterando alguns valores
valores_ausentes = ["***","###!","****","*****","NULL"]
df = pd.read_csv("./ocorrencia.csv", sep=";", parse_dates=["ocorrencia_dia"], dayfirst=True, na_values=valores_ausentes)
df.head(10)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
5,39807,39807,INCIDENTE,SALVADOR,BA,NaN,2010-01-06,17:53:00,0
6,40215,40215,INCIDENTE,COARI,AM,SBUY,2010-01-07,18:40:00,0
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0


Usando pandora

O schema levanta error parando a execução quando algum item nao passa na validação

In [4]:
# montando schema
schema = pa.DataFrameSchema(
    columns = {
        "codigo":pa.Column(pa.Int, required=False),# o required diz que a coluna pode ou nao ta no dataframe
        "codigo_ocorrencia":pa.Column(pa.Int),
        "codigo_ocorrencia2":pa.Column(pa.Int),
        "ocorrencia_classificacao":pa.Column(pa.String),
        "ocorrencia_cidade":pa.Column(pa.String),
        "ocorrencia_uf":pa.Column(pa.String, pa.Check.str_length(2,2),nullable=True), # definindo o tamanho do dado
        "ocorrencia_aerodromo":pa.Column(pa.String, nullable=True),
        "ocorrencia_dia":pa.Column(pa.DateTime), # Check.str_matches, usa regex para checa se o formato de hora esta correto
        "ocorrencia_hora": pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'), nullable=True),# permite que a coluna tenha valores Null
        "total_recomendacoes": pa.Column(pa.Int)
        
    }
)

In [5]:
schema.validate(df) # o error vai ser resolvido na etapa de limpeza de dados

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
6109,80245,80245,INCIDENTE,RECIFE,PE,SBRF,2021-08-15,15:05:00,0
6110,80247,80247,INCIDENTE,PORTO VELHO,RO,SBPV,2021-08-16,02:30:00,0
6111,80251,80251,INCIDENTE,BELÉM,PA,SBBE,2021-08-17,15:45:00,0
6112,80257,80257,ACIDENTE,DOM AQUINO,MT,NaN,2021-08-18,16:00:00,0


In [6]:
# verificando os tipos de dados
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

In [7]:
# fazendo buscas
df.loc[1]

codigo_ocorrencia                         40349
codigo_ocorrencia2                        40349
ocorrencia_classificacao              INCIDENTE
ocorrencia_cidade                         BELÉM
ocorrencia_uf                                PA
ocorrencia_aerodromo                       SBBE
ocorrencia_dia              2010-01-03 00:00:00
ocorrencia_hora                        11:05:00
total_recomendacoes                           0
Name: 1, dtype: object

In [ ]:
# usando iloc, pode usar o index para faze a busca
df.iloc[1]

In [9]:
# usando iloc, pode usar o index para faze a busca
df.iloc[-1] # pengando o ultimo dado do dataframe

codigo_ocorrencia                         80259
codigo_ocorrencia2                        80259
ocorrencia_classificacao              INCIDENTE
ocorrencia_cidade                      CAMPINAS
ocorrencia_uf                                SP
ocorrencia_aerodromo                       SBKP
ocorrencia_dia              2021-08-18 00:00:00
ocorrencia_hora                        12:45:00
total_recomendacoes                           0
Name: 6113, dtype: object

In [10]:
# tambem é possivel fazer fatiamento usando o label
df.iloc[10:15]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
10,39789,39789,INCIDENTE,SÃO PEDRO DO SUL,RS,NaN,2010-01-10,21:30:00,0
11,40069,40069,ACIDENTE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8
12,40414,40414,INCIDENTE,BRASÍLIA,DF,SBBR,2010-01-10,03:00:00,0
13,39507,39507,ACIDENTE,PRIMAVERA DO LESTE,MT,NaN,2010-01-11,17:00:00,2
14,40107,40107,ACIDENTE,GURUPÁ,PA,SNGU,2010-01-12,14:30:00,0


In [11]:
df.loc[:,"ocorrencia_uf"] # ou

0       RJ
1       PA
2       RJ
3       MT
4       RS
        ..
6109    PE
6110    RO
6111    PA
6112    MT
6113    SP
Name: ocorrencia_uf, Length: 6114, dtype: object

In [12]:
df["ocorrencia_uf"] 

0       RJ
1       PA
2       RJ
3       MT
4       RS
        ..
6109    PE
6110    RO
6111    PA
6112    MT
6113    SP
Name: ocorrencia_uf, Length: 6114, dtype: object

In [13]:
# verificando a quantidade de valores na(nao informados)
df.isna().sum()

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  2
ocorrencia_aerodromo        2443
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [14]:
# verificando a quantidade de valores na(nao informados)
df.isnull().sum()

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  2
ocorrencia_aerodromo        2443
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [15]:
# buscando os valores null na coluna ocorrencia_uf
# df.loc[df.ocorrencia_uf.isnull()]
filtro = df.ocorrencia_uf.isnull()
df.loc[filtro]


,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
2227,49474,49474,ACIDENTE,ÁGUAS INTERNACIONAIS,NaN,NaN,2013-09-02,02:54:00,0
5855,79956,79956,ACIDENTE,NÃO IDENTIFICADA,NaN,NaN,2021-02-26,11:00:00,0


In [16]:
# buscando os valores null na coluna ocorrencia_hora
# df.loc[df.ocorrencia_hora.isnull()]
filtro = df.ocorrencia_hora.isnull()
df.loc[filtro]


,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
4102,78823,78823,INCIDENTE,CORUMBÁ,MS,NaN,2017-07-09,NaN,0


In [17]:
# usando o count, ele pode mostra a colunas que tem os valores null
df.count()

codigo_ocorrencia           6114
codigo_ocorrencia2          6114
ocorrencia_classificacao    6114
ocorrencia_cidade           6114
ocorrencia_uf               6112
ocorrencia_aerodromo        3671
ocorrencia_dia              6114
ocorrencia_hora             6113
total_recomendacoes         6114
dtype: int64

In [18]:
# ocorrencias com mias de 10 recomendações
filtro = df.total_recomendacoes > 10
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
63,39992,39992,ACIDENTE,SÃO PAULO,SP,SBMT,2010-02-17,20:07:00,11
326,42250,42250,ACIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-08-12,12:26:00,19
530,43471,43471,ACIDENTE,SÃO PAULO,SP,NaN,2010-12-14,20:15:00,11
727,44377,44377,ACIDENTE,ITÁPOLIS,SP,SDIO,2011-04-09,14:50:00,11
872,44796,44796,ACIDENTE,RECIFE,PE,SBRF,2011-07-13,09:54:00,23
884,44888,44888,INCIDENTE GRAVE,SÃO JOSÉ DOS PINHAIS,PR,SBCT,2011-07-15,17:05:00,11
1062,45554,45554,INCIDENTE GRAVE,UBERLÂNDIA,MG,SBUL,2011-11-15,02:15:00,20
1972,47938,47938,INCIDENTE,BRASÍLIA,DF,SBBR,2013-04-13,18:00:00,11
2797,52265,52265,ACIDENTE,SANTOS,SP,NaN,2014-08-13,13:03:00,13
3932,66432,66432,INCIDENTE GRAVE,VITÓRIA,ES,NaN,2017-02-21,11:47:00,12


In [19]:
# ocorrencias com mais de 10 recomendações e as cidades
filtro = df.total_recomendacoes > 10
df.loc[filtro, ['ocorrencia_cidade','total_recomendacoes']]

,ocorrencia_cidade,total_recomendacoes
63,SÃO PAULO,11
326,RIO DE JANEIRO,19
530,SÃO PAULO,11
727,ITÁPOLIS,11
872,RECIFE,23
884,SÃO JOSÉ DOS PINHAIS,11
1062,UBERLÂNDIA,20
1972,BRASÍLIA,11
2797,SANTOS,13
3932,VITÓRIA,12


In [20]:
# ocorrencias  cim as classificaçoes iguais a INCIDENTE GRAVE
filtro = df.ocorrencia_classificacao == "INCIDENTE GRAVE"
df.loc[filtro, ['ocorrencia_cidade','ocorrencia_classificacao']]

,ocorrencia_cidade,ocorrencia_classificacao
7,CANUTAMA,INCIDENTE GRAVE
8,CASCAVEL,INCIDENTE GRAVE
9,PARÁ DE MINAS,INCIDENTE GRAVE
29,CURITIBA,INCIDENTE GRAVE
35,PALMAS,INCIDENTE GRAVE
...,...,...
6090,MARAÚ,INCIDENTE GRAVE
6095,ELDORADO DO SUL,INCIDENTE GRAVE
6097,CAXIAS DO SUL,INCIDENTE GRAVE
6098,CHAPADÃO DO SUL,INCIDENTE GRAVE


In [21]:
# ocorrencias  cim as classificaçoes iguais a INCIDENTE GRAVE e o estado == SP
filtro = df.ocorrencia_classificacao == "INCIDENTE GRAVE"
filtro2 = df.ocorrencia_uf == "SP"
#df.loc[filtro & filtro2, ['ocorrencia_uf','ocorrencia_classificacao']]
df.loc[filtro & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
136,40270,40270,INCIDENTE GRAVE,PIRASSUNUNGA,SP,SDPY,2010-04-15,20:15:00,0
171,40620,40620,INCIDENTE GRAVE,GUARULHOS,SP,SBGR,2010-05-06,15:50:00,0
219,41411,41411,INCIDENTE GRAVE,SÃO PAULO,SP,SBMT,2010-06-08,19:30:00,2
297,42289,42289,INCIDENTE GRAVE,SÃO PAULO,SP,SBMT,2010-07-29,20:06:00,0
314,42323,42323,INCIDENTE GRAVE,SÃO JOSÉ DO RIO PRETO,SP,SBSR,2010-08-05,12:01:00,0
...,...,...,...,...,...,...,...,...,...
5949,80056,80056,INCIDENTE GRAVE,FERNANDÓPOLIS,SP,NaN,2021-04-28,11:20:00,0
6000,80114,80114,INCIDENTE GRAVE,LINS,SP,SWXQ,2021-06-05,19:43:00,0
6007,80124,80124,INCIDENTE GRAVE,SÃO PAULO,SP,SBSP,2021-06-10,20:30:00,0
6081,80200,80200,INCIDENTE GRAVE,BRAGANÇA PAULISTA,SP,SDVH,2021-07-25,12:25:00,0


In [22]:
# ocorrencias  cim as classificaçoes iguais a INCIDENTE GRAVE ou estado == SP
filtro = df.ocorrencia_classificacao == "INCIDENTE GRAVE"
filtro2 = df.ocorrencia_uf == "SP"
#df.loc[filtro & filtro2, ['ocorrencia_uf','ocorrencia_classificacao']]
df.loc[filtro | filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0
11,40069,40069,ACIDENTE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8
16,39809,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0
...,...,...,...,...,...,...,...,...,...
6097,80231,80231,INCIDENTE GRAVE,CAXIAS DO SUL,RS,SBCX,2021-08-06,16:59:00,0
6098,80248,80248,INCIDENTE GRAVE,CHAPADÃO DO SUL,MS,SSCD,2021-08-06,20:00:00,0
6104,80238,80238,INCIDENTE GRAVE,VOTUPORANGA,SP,SDVG,2021-08-11,19:09:00,0
6108,80244,80244,INCIDENTE,BRAGANÇA PAULISTA,SP,SBBP,2021-08-15,15:20:00,0


In [23]:
# ocorrencias cuja (classificação == INCIDENTE GRAVE ou  classificação == INCIDENTE) e o estado == SP
filtro = df.ocorrencia_classificacao == "INCIDENTE GRAVE"
filtro1 = df.ocorrencia_classificacao == "INCIDENTE"
filtro2 = df.ocorrencia_uf == "SP"
#df.loc[filtro & filtro2, ['ocorrencia_uf','ocorrencia_classificacao']]
df.loc[filtro | filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0
16,39809,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0
18,39828,39828,INCIDENTE,SANTOS,SP,NaN,2010-01-15,17:45:00,0
...,...,...,...,...,...,...,...,...,...
6097,80231,80231,INCIDENTE GRAVE,CAXIAS DO SUL,RS,SBCX,2021-08-06,16:59:00,0
6098,80248,80248,INCIDENTE GRAVE,CHAPADÃO DO SUL,MS,SSCD,2021-08-06,20:00:00,0
6104,80238,80238,INCIDENTE GRAVE,VOTUPORANGA,SP,SDVG,2021-08-11,19:09:00,0
6108,80244,80244,INCIDENTE,BRAGANÇA PAULISTA,SP,SBBP,2021-08-15,15:20:00,0


In [24]:
# ocorrencias cuja (classificação == INCIDENTE GRAVE ou  classificação == INCIDENTE) e o estado == SP
filtro = (df.ocorrencia_classificacao == "INCIDENTE GRAVE") | (df.ocorrencia_classificacao == "INCIDENTE")
filtro2 = df.ocorrencia_uf == "SP"
#df.loc[filtro & filtro2, ['ocorrencia_uf','ocorrencia_classificacao']]
df.loc[filtro & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
16,39809,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0
18,39828,39828,INCIDENTE,SANTOS,SP,NaN,2010-01-15,17:45:00,0
26,39847,39847,INCIDENTE,SOROCABA,SP,NaN,2010-01-20,13:10:00,0
27,39768,39768,INCIDENTE,CAMPINAS,SP,NaN,2010-01-21,20:45:00,0
33,39848,39848,INCIDENTE,GUARULHOS,SP,SBGR,2010-01-25,17:05:00,0
...,...,...,...,...,...,...,...,...,...
6084,80234,80234,INCIDENTE,SÃO PAULO,SP,SBSP,2021-07-26,13:20:00,0
6085,80208,80208,INCIDENTE,SÃO PAULO,SP,SBMT,2021-07-27,19:45:00,0
6104,80238,80238,INCIDENTE GRAVE,VOTUPORANGA,SP,SDVG,2021-08-11,19:09:00,0
6108,80244,80244,INCIDENTE,BRAGANÇA PAULISTA,SP,SBBP,2021-08-15,15:20:00,0


In [25]:
# ocorrencias cuja (classificação == INCIDENTE GRAVE ou  classificação == INCIDENTE) e o estado == SP
filtro = df.ocorrencia_classificacao.isin(["INCIDENTE GRAVE","INCIDENTE"])
filtro2 = df.ocorrencia_uf == "SP"
#df.loc[filtro & filtro2, ['ocorrencia_uf','ocorrencia_classificacao']]
df.loc[filtro & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
16,39809,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0
18,39828,39828,INCIDENTE,SANTOS,SP,NaN,2010-01-15,17:45:00,0
26,39847,39847,INCIDENTE,SOROCABA,SP,NaN,2010-01-20,13:10:00,0
27,39768,39768,INCIDENTE,CAMPINAS,SP,NaN,2010-01-21,20:45:00,0
33,39848,39848,INCIDENTE,GUARULHOS,SP,SBGR,2010-01-25,17:05:00,0
...,...,...,...,...,...,...,...,...,...
6084,80234,80234,INCIDENTE,SÃO PAULO,SP,SBSP,2021-07-26,13:20:00,0
6085,80208,80208,INCIDENTE,SÃO PAULO,SP,SBMT,2021-07-27,19:45:00,0
6104,80238,80238,INCIDENTE GRAVE,VOTUPORANGA,SP,SDVG,2021-08-11,19:09:00,0
6108,80244,80244,INCIDENTE,BRAGANÇA PAULISTA,SP,SBBP,2021-08-15,15:20:00,0


In [26]:
# ocorrencias cujas cidades comecem com a letra C
filtro = df.ocorrencia_cidade.str[0] == "C"
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
6,40215,40215,INCIDENTE,COARI,AM,SBUY,2010-01-07,18:40:00,0
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
15,39315,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
20,40310,40310,INCIDENTE,CRUZEIRO DO SUL,AC,SBCZ,2010-01-18,19:32:00,0
...,...,...,...,...,...,...,...,...,...
6094,80225,80225,INCIDENTE,CURITIBA,PR,SBBI,2021-08-04,13:45:00,0
6097,80231,80231,INCIDENTE GRAVE,CAXIAS DO SUL,RS,SBCX,2021-08-06,16:59:00,0
6098,80248,80248,INCIDENTE GRAVE,CHAPADÃO DO SUL,MS,SSCD,2021-08-06,20:00:00,0
6100,80235,80235,INCIDENTE,CORUMBÁ,MS,SBCR,2021-08-09,20:55:00,0


In [27]:
# ocorrencias cujas cidades comecem com a letra A
filtro = df.ocorrencia_cidade.str[-1] == "A"
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
12,40414,40414,INCIDENTE,BRASÍLIA,DF,SBBR,2010-01-10,03:00:00,0
15,39315,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
25,39772,39772,INCIDENTE,UBERLÂNDIA,MG,NaN,2010-01-20,18:58:00,0
26,39847,39847,INCIDENTE,SOROCABA,SP,NaN,2010-01-20,13:10:00,0
...,...,...,...,...,...,...,...,...,...
6093,80227,80227,ACIDENTE,UMUARAMA,PR,NaN,2021-08-02,14:30:00,0
6094,80225,80225,INCIDENTE,CURITIBA,PR,SBBI,2021-08-04,13:45:00,0
6102,80240,80240,INCIDENTE,GOIÂNIA,GO,SBNV,2021-08-10,14:15:00,0
6104,80238,80238,INCIDENTE GRAVE,VOTUPORANGA,SP,SDVG,2021-08-11,19:09:00,0


In [28]:
# ocorrencias cujas cidades termina com os caracteres MA
filtro = df.ocorrencia_cidade.str[-2:] == "MA"
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
15,39315,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
408,43062,43062,ACIDENTE,NOVO GAMA,GO,NaN,2010-09-25,16:30:00,1
688,44198,44198,ACIDENTE,BURITAMA,SP,NaN,2011-03-19,11:18:00,0
1035,51785,51785,INCIDENTE,BURITAMA,SP,SIBX,2011-10-25,19:15:00,0
1377,45939,45939,ACIDENTE,UMUARAMA,PR,NaN,2012-05-30,19:00:00,0
1401,46180,46180,ACIDENTE,AURIFLAMA,SP,NaN,2012-06-16,19:30:00,0
2099,49113,49113,INCIDENTE GRAVE,NOVA LIMA,MG,NaN,2013-06-15,12:00:00,4
2102,48799,48799,ACIDENTE,CANUTAMA,AM,NaN,2013-06-18,21:30:00,0
2646,80059,80059,INCIDENTE,IVINHEMA,MS,NaN,2014-04-27,11:30:00,0


In [29]:
# ocorrencias cujas cidades contem(em qualquer parte do conteudo) com os caracteres MA ou AL
filtro = df.ocorrencia_cidade.str.contains("MA|AL")
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
5,39807,39807,INCIDENTE,SALVADOR,BA,NaN,2010-01-06,17:53:00,0
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
13,39507,39507,ACIDENTE,PRIMAVERA DO LESTE,MT,NaN,2010-01-11,17:00:00,2
15,39315,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
17,39810,39810,INCIDENTE,SALVADOR,BA,NaN,2010-01-15,18:23:00,0
...,...,...,...,...,...,...,...,...,...
6075,80205,80205,INCIDENTE,MANAUS,AM,SBEG,2021-07-23,11:15:00,0
6086,80212,80212,INCIDENTE,AMAPÁ,AP,NaN,2021-07-28,18:00:00,0
6090,80221,80221,INCIDENTE GRAVE,MARAÚ,BA,SIRI,2021-07-31,13:15:00,0
6093,80227,80227,ACIDENTE,UMUARAMA,PR,NaN,2021-08-02,14:30:00,0


In [30]:
# ocorrencias do ano de 2015 
filtro = df.ocorrencia_dia.dt.year == 2015
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
2998,52992,52992,INCIDENTE GRAVE,SALVADOR,BA,SBSV,2015-01-01,11:40:00,3
2999,52979,52979,ACIDENTE,IVINHEMA,MS,NaN,2015-01-02,14:50:00,0
3000,53073,53073,INCIDENTE,TEFÉ,AM,SBTF,2015-01-02,16:28:00,0
3001,53074,53074,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-01-02,20:34:00,0
3002,52976,52976,ACIDENTE,TOLEDO,PR,SBTD,2015-01-04,22:04:00,6
...,...,...,...,...,...,...,...,...,...
3464,60632,60632,INCIDENTE GRAVE,ITABERÁ,SP,NaN,2015-12-24,14:00:00,0
3465,60600,60600,INCIDENTE,GUARULHOS,SP,SBGR,2015-12-25,19:00:00,0
3466,60642,60642,INCIDENTE,SÃO FRANCISCO DO SUL,SC,SSSS,2015-12-26,16:00:00,0
3467,60631,60631,ACIDENTE,MAÇAMBARÁ,RS,NaN,2015-12-28,19:00:00,2


In [31]:
# ocorrencias do ano de 2015 e mes 12
filtro = df.ocorrencia_dia.dt.year == 2015
filtro2 = df.ocorrencia_dia.dt.month == 12
df.loc[filtro & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
3430,53573,53573,INCIDENTE,GUARULHOS,SP,SBGR,2015-12-01,02:48:00,0
3431,60601,60601,INCIDENTE,PALMAS,TO,SBPJ,2015-12-01,16:05:00,0
3432,53634,53634,INCIDENTE,PALMAS,TO,SBPJ,2015-12-02,17:45:00,0
3433,53636,53636,INCIDENTE,JUNDIAÍ,SP,SBJD,2015-12-02,17:42:00,0
3434,53575,53575,INCIDENTE,CAMPOS DOS GOYTACAZES,RJ,SBFS,2015-12-03,10:50:00,0
3435,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0
3436,53625,53625,ACIDENTE,TRINDADE,GO,NaN,2015-12-06,13:10:00,3
3437,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1
3438,53628,53628,ACIDENTE,AGUAÍ,SP,NaN,2015-12-08,14:30:00,1
3439,53629,53629,INCIDENTE GRAVE,JALES,SP,SDJL,2015-12-08,10:20:00,0


In [32]:
# ocorrencias do ano de 2015 e mes 12
filtro = (df.ocorrencia_dia.dt.year == 2015) & (df.ocorrencia_dia.dt.month == 12)
df.loc[filtro & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
3430,53573,53573,INCIDENTE,GUARULHOS,SP,SBGR,2015-12-01,02:48:00,0
3431,60601,60601,INCIDENTE,PALMAS,TO,SBPJ,2015-12-01,16:05:00,0
3432,53634,53634,INCIDENTE,PALMAS,TO,SBPJ,2015-12-02,17:45:00,0
3433,53636,53636,INCIDENTE,JUNDIAÍ,SP,SBJD,2015-12-02,17:42:00,0
3434,53575,53575,INCIDENTE,CAMPOS DOS GOYTACAZES,RJ,SBFS,2015-12-03,10:50:00,0
3435,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0
3436,53625,53625,ACIDENTE,TRINDADE,GO,NaN,2015-12-06,13:10:00,3
3437,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1
3438,53628,53628,ACIDENTE,AGUAÍ,SP,NaN,2015-12-08,14:30:00,1
3439,53629,53629,INCIDENTE GRAVE,JALES,SP,SDJL,2015-12-08,10:20:00,0


In [33]:
# ocorrencias do ano de 2015 e mes 12, dia 8
filtro = (df.ocorrencia_dia.dt.year == 2015) & (df.ocorrencia_dia.dt.month == 12) & (df.ocorrencia_dia.dt.day == 8)
df.loc[filtro & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
3438,53628,53628,ACIDENTE,AGUAÍ,SP,NaN,2015-12-08,14:30:00,1
3439,53629,53629,INCIDENTE GRAVE,JALES,SP,SDJL,2015-12-08,10:20:00,0
3440,53631,53631,INCIDENTE,CAMPINAS,SP,SBKP,2015-12-08,16:19:00,0
3441,60636,60636,INCIDENTE,CAXIAS DO SUL,RS,SBCX,2015-12-08,13:00:00,0


In [34]:
# ocorrencias do ano de 2015 e mes 12, dia entre 3 e 8
filtro_ano = df.ocorrencia_dia.dt.year == 2015
filtro_mes = df.ocorrencia_dia.dt.month == 12
filtro_dia = (df.ocorrencia_dia.dt.day >=3) & (df.ocorrencia_dia.dt.day <= 8)
df.loc[filtro_ano & filtro_mes & filtro_dia]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
3434,53575,53575,INCIDENTE,CAMPOS DOS GOYTACAZES,RJ,SBFS,2015-12-03,10:50:00,0
3435,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0
3436,53625,53625,ACIDENTE,TRINDADE,GO,NaN,2015-12-06,13:10:00,3
3437,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1
3438,53628,53628,ACIDENTE,AGUAÍ,SP,NaN,2015-12-08,14:30:00,1
3439,53629,53629,INCIDENTE GRAVE,JALES,SP,SDJL,2015-12-08,10:20:00,0
3440,53631,53631,INCIDENTE,CAMPINAS,SP,SBKP,2015-12-08,16:19:00,0
3441,60636,60636,INCIDENTE,CAXIAS DO SUL,RS,SBCX,2015-12-08,13:00:00,0


In [35]:
# criando nova coluna que vai conter dia e hora
df["ocorrencia_dia_hora"] = pd.to_datetime(df.ocorrencia_dia.astype(str) + " " + df.ocorrencia_hora) 

In [36]:
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
ocorrencia_dia_hora         datetime64[ns]
dtype: object

In [37]:
df["ocorrencia_dia_hora"]

0      2010-01-03 12:00:00
1      2010-01-03 11:05:00
2      2010-01-03 03:00:00
3      2010-01-04 17:30:00
4      2010-01-05 19:25:00
               ...        
6109   2021-08-15 15:05:00
6110   2021-08-16 02:30:00
6111   2021-08-17 15:45:00
6112   2021-08-18 16:00:00
6113   2021-08-18 12:45:00
Name: ocorrencia_dia_hora, Length: 6114, dtype: datetime64[ns]

In [38]:
# ocorrencias do ano de 2015 e mes 12, dia entre 3 e 8
filtro_ano = df.ocorrencia_dia_hora.dt.year == 2015
filtro_mes = df.ocorrencia_dia_hora.dt.month == 12
filtro_dia = (df.ocorrencia_dia_hora.dt.day >=3) & (df.ocorrencia_dia_hora.dt.day <= 8)
df.loc[filtro_ano & filtro_mes & filtro_dia]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
3434,53575,53575,INCIDENTE,CAMPOS DOS GOYTACAZES,RJ,SBFS,2015-12-03,10:50:00,0,2015-12-03 10:50:00
3435,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0,2015-12-03 16:47:00
3436,53625,53625,ACIDENTE,TRINDADE,GO,NaN,2015-12-06,13:10:00,3,2015-12-06 13:10:00
3437,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1,2015-12-06 15:00:00
3438,53628,53628,ACIDENTE,AGUAÍ,SP,NaN,2015-12-08,14:30:00,1,2015-12-08 14:30:00
3439,53629,53629,INCIDENTE GRAVE,JALES,SP,SDJL,2015-12-08,10:20:00,0,2015-12-08 10:20:00
3440,53631,53631,INCIDENTE,CAMPINAS,SP,SBKP,2015-12-08,16:19:00,0,2015-12-08 16:19:00
3441,60636,60636,INCIDENTE,CAXIAS DO SUL,RS,SBCX,2015-12-08,13:00:00,0,2015-12-08 13:00:00


In [39]:
filtro1 = df.ocorrencia_dia_hora > "2015-12-03 10:50:00"
df.loc[filtro1]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
3435,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0,2015-12-03 16:47:00
3436,53625,53625,ACIDENTE,TRINDADE,GO,NaN,2015-12-06,13:10:00,3,2015-12-06 13:10:00
3437,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1,2015-12-06 15:00:00
3438,53628,53628,ACIDENTE,AGUAÍ,SP,NaN,2015-12-08,14:30:00,1,2015-12-08 14:30:00
3439,53629,53629,INCIDENTE GRAVE,JALES,SP,SDJL,2015-12-08,10:20:00,0,2015-12-08 10:20:00
...,...,...,...,...,...,...,...,...,...,...
6109,80245,80245,INCIDENTE,RECIFE,PE,SBRF,2021-08-15,15:05:00,0,2021-08-15 15:05:00
6110,80247,80247,INCIDENTE,PORTO VELHO,RO,SBPV,2021-08-16,02:30:00,0,2021-08-16 02:30:00
6111,80251,80251,INCIDENTE,BELÉM,PA,SBBE,2021-08-17,15:45:00,0,2021-08-17 15:45:00
6112,80257,80257,ACIDENTE,DOM AQUINO,MT,NaN,2021-08-18,16:00:00,0,2021-08-18 16:00:00


In [40]:
filtro1 = df.ocorrencia_dia_hora >= "2015-12-03 10:50:00"
filtro2 = df.ocorrencia_dia_hora <= "2015-12-08 10:50:00"
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
3434,53575,53575,INCIDENTE,CAMPOS DOS GOYTACAZES,RJ,SBFS,2015-12-03,10:50:00,0,2015-12-03 10:50:00
3435,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0,2015-12-03 16:47:00
3436,53625,53625,ACIDENTE,TRINDADE,GO,NaN,2015-12-06,13:10:00,3,2015-12-06 13:10:00
3437,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1,2015-12-06 15:00:00
3439,53629,53629,INCIDENTE GRAVE,JALES,SP,SDJL,2015-12-08,10:20:00,0,2015-12-08 10:20:00


In [41]:
# ocorrencias do ano de 2015 e mes 03
filtro1 = df.ocorrencia_dia.dt.year == 2015
filtro2 = df.ocorrencia_dia.dt.month == 3
df201503 = df.loc[filtro1 & filtro2]
df201503

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
3088,53183,53183,ACIDENTE,AMAPORÃ,PR,NaN,2015-03-02,23:00:00,0,2015-03-02 23:00:00
3089,53120,53120,ACIDENTE,CHAVES,PA,SNXW,2015-03-04,13:30:00,0,2015-03-04 13:30:00
3090,53109,53109,ACIDENTE,CAMPO GRANDE,MS,SSIE,2015-03-05,13:50:00,0,2015-03-05 13:50:00
3091,53112,53112,ACIDENTE,MOGI GUAÇU,SP,NaN,2015-03-06,21:00:00,0,2015-03-06 21:00:00
3092,53152,53152,INCIDENTE GRAVE,RIO DE JANEIRO,RJ,SBRJ,2015-03-10,11:30:00,2,2015-03-10 11:30:00
3093,53167,53167,INCIDENTE,MARABÁ,PA,SBMA,2015-03-10,17:33:00,0,2015-03-10 17:33:00
3094,53596,53596,INCIDENTE,ITAPEMA,SC,NaN,2015-03-10,21:35:00,0,2015-03-10 21:35:00
3095,53149,53149,ACIDENTE,TABATINGA,AM,SBTT,2015-03-11,23:25:00,0,2015-03-11 23:25:00
3096,53148,53148,ACIDENTE,ARARAS,SP,SDEH,2015-03-12,12:40:00,0,2015-03-12 12:40:00
3097,53153,53153,INCIDENTE,PORTO ALEGRE,RS,SBPA,2015-03-13,15:45:00,0,2015-03-13 15:45:00


In [42]:
df201503

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
3088,53183,53183,ACIDENTE,AMAPORÃ,PR,NaN,2015-03-02,23:00:00,0,2015-03-02 23:00:00
3089,53120,53120,ACIDENTE,CHAVES,PA,SNXW,2015-03-04,13:30:00,0,2015-03-04 13:30:00
3090,53109,53109,ACIDENTE,CAMPO GRANDE,MS,SSIE,2015-03-05,13:50:00,0,2015-03-05 13:50:00
3091,53112,53112,ACIDENTE,MOGI GUAÇU,SP,NaN,2015-03-06,21:00:00,0,2015-03-06 21:00:00
3092,53152,53152,INCIDENTE GRAVE,RIO DE JANEIRO,RJ,SBRJ,2015-03-10,11:30:00,2,2015-03-10 11:30:00
3093,53167,53167,INCIDENTE,MARABÁ,PA,SBMA,2015-03-10,17:33:00,0,2015-03-10 17:33:00
3094,53596,53596,INCIDENTE,ITAPEMA,SC,NaN,2015-03-10,21:35:00,0,2015-03-10 21:35:00
3095,53149,53149,ACIDENTE,TABATINGA,AM,SBTT,2015-03-11,23:25:00,0,2015-03-11 23:25:00
3096,53148,53148,ACIDENTE,ARARAS,SP,SDEH,2015-03-12,12:40:00,0,2015-03-12 12:40:00
3097,53153,53153,INCIDENTE,PORTO ALEGRE,RS,SBPA,2015-03-13,15:45:00,0,2015-03-13 15:45:00


In [43]:
# agrupamento
df201503.groupby(['codigo_ocorrencia']).codigo_ocorrencia.count()

codigo_ocorrencia
53109    1
53112    1
53120    1
53148    1
53149    1
53151    1
53152    1
53153    1
53154    1
53164    1
53165    1
53166    1
53167    1
53168    1
53169    1
53170    1
53178    1
53179    1
53180    1
53181    1
53182    1
53183    1
53184    1
53185    1
53186    1
53187    1
53188    1
53189    1
53190    1
53193    1
53194    1
53211    1
53212    1
53213    1
53238    1
53412    1
53596    1
Name: codigo_ocorrencia, dtype: int64

In [44]:
# agrupamento
df201503.groupby(['ocorrencia_classificacao']).codigo_ocorrencia.count()

ocorrencia_classificacao
ACIDENTE           15
INCIDENTE          17
INCIDENTE GRAVE     5
Name: codigo_ocorrencia, dtype: int64

In [45]:
# agrupamento
df201503.groupby(['ocorrencia_classificacao']).size()

ocorrencia_classificacao
ACIDENTE           15
INCIDENTE          17
INCIDENTE GRAVE     5
dtype: int64

In [46]:
# agrupamento
df201503.groupby(['ocorrencia_classificacao']).size().sort_values()

ocorrencia_classificacao
INCIDENTE GRAVE     5
ACIDENTE           15
INCIDENTE          17
dtype: int64

In [47]:
# agrupamento
df201503.groupby(['ocorrencia_classificacao']).size().sort_values(ascending=False)

ocorrencia_classificacao
INCIDENTE          17
ACIDENTE           15
INCIDENTE GRAVE     5
dtype: int64

In [48]:
filtro1 = df.ocorrencia_dia.dt.year == 2010
filtro2 = df.ocorrencia_uf.isin(["SP","MG","ES", "RJ"])
dfsudeste2010 = df.loc[filtro1 & filtro2]
dfsudeste2010

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0,2010-01-03 12:00:00
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0,2010-01-03 03:00:00
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0,2010-01-10 20:00:00
11,40069,40069,ACIDENTE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8,2010-01-10 14:50:00
16,39809,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0,2010-01-15 15:00:00
...,...,...,...,...,...,...,...,...,...,...
541,43722,43722,ACIDENTE,CAJOBI,SP,NaN,2010-12-23,11:00:00,0,2010-12-23 11:00:00
544,43548,43548,INCIDENTE,SÃO PAULO,SP,SBSP,2010-12-26,22:10:00,0,2010-12-26 22:10:00
548,43724,43724,ACIDENTE,COSMÓPOLIS,SP,NaN,2010-12-29,12:00:00,0,2010-12-29 12:00:00
550,43730,43730,INCIDENTE,VITÓRIA,ES,NaN,2010-12-30,18:10:00,0,2010-12-30 18:10:00


In [49]:
# agrupamento
dfsudeste2010.groupby(["ocorrencia_classificacao"]).size()

ocorrencia_classificacao
ACIDENTE            40
INCIDENTE          170
INCIDENTE GRAVE     18
dtype: int64

In [50]:
# agrupamento
dfsudeste2010.groupby(["ocorrencia_classificacao","ocorrencia_uf"]).size()

ocorrencia_classificacao  ocorrencia_uf
ACIDENTE                  MG                7
                          RJ                9
                          SP               24
INCIDENTE                 ES                5
                          MG               39
                          RJ               52
                          SP               74
INCIDENTE GRAVE           MG                5
                          RJ                4
                          SP                9
dtype: int64

In [51]:
# agrupamento
dfsudeste2010.groupby(["ocorrencia_cidade"]).size().sort_values(ascending=False)

ocorrencia_cidade
RIO DE JANEIRO           47
SÃO PAULO                33
GUARULHOS                18
BELO HORIZONTE           17
CAMPINAS                 13
MACAÉ                    12
CONFINS                   7
UBERLÂNDIA                5
VITÓRIA                   4
JUIZ DE FORA              4
SÃO JOSÉ DO RIO PRETO     3
SÃO JOSÉ DOS CAMPOS       3
MARÍLIA                   3
ARARAQUARA                2
MONTES CLAROS             2
CAMPOS DOS GOYTACAZES     2
DIVINÓPOLIS               2
IPATINGA                  2
ITANHAÉM                  2
JUNDIAÍ                   2
POÇOS DE CALDAS           2
RIBEIRÃO PRETO            2
SOROCABA                  2
UBERABA                   2
BAURU                     2
AREALVA                   2
BOITUVA                   1
BOCAINA                   1
BOM JESUS DO AMPARO       1
BOM JESUS DO GALHO        1
BEBEDOURO                 1
BRAGANÇA PAULISTA         1
BROTAS                    1
CAJOBI                    1
ARRAIAL DO CABO           1
CA

In [52]:
# agrupamento
filtro = dfsudeste2010.ocorrencia_cidade == "RIO DE JANEIRO"
dfsudeste2010.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0,2010-01-03 12:00:00
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0,2010-01-03 03:00:00
19,44575,44575,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2010-01-17,12:27:00,0,2010-01-17 12:27:00
23,40311,40311,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-19,13:20:00,0,2010-01-19 13:20:00
24,41577,41577,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-19,12:16:00,0,2010-01-19 12:16:00
30,39849,39849,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2010-01-23,03:30:00,0,2010-01-23 03:30:00
39,40617,40617,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2010-01-27,20:06:00,0,2010-01-27 20:06:00
49,39295,39295,ACIDENTE,RIO DE JANEIRO,RJ,NaN,2010-02-06,21:09:00,0,2010-02-06 21:09:00
71,39987,39987,INCIDENTE,RIO DE JANEIRO,RJ,SBJR,2010-02-24,19:37:00,0,2010-02-24 19:37:00
101,40210,40210,INCIDENTE,RIO DE JANEIRO,RJ,SBJR,2010-03-20,20:02:00,0,2010-03-20 20:02:00


In [53]:
# somando o total de recomendacoes que o rio de janeiro teve
filtro = dfsudeste2010.ocorrencia_cidade == "RIO DE JANEIRO"
dfsudeste2010.loc[filtro].total_recomendacoes.sum()

25

In [54]:
# somando o total de recomendacoes que o rio de janeiro teve
filtro = dfsudeste2010.ocorrencia_cidade == "RIO DE JANEIRO"
filtro2 = dfsudeste2010.total_recomendacoes > 0
dfsudeste2010.loc[filtro & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
119,40190,40190,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2010-04-06,02:35:00,3,2010-04-06 02:35:00
274,41750,41750,ACIDENTE,RIO DE JANEIRO,RJ,SBJR,2010-07-12,19:53:00,3,2010-07-12 19:53:00
326,42250,42250,ACIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-08-12,12:26:00,19,2010-08-12 12:26:00


In [55]:
# totais de recomendações pode cidade
dfsudeste2010.groupby(["ocorrencia_cidade"]).total_recomendacoes.sum()

ocorrencia_cidade
AMERICANA                 0
ARARAQUARA                0
ARAÇATUBA                 0
AREALVA                   0
ARRAIAL DO CABO           0
BAURU                     0
BEBEDOURO                 0
BELO HORIZONTE            2
BOCAINA                   0
BOITUVA                   1
BOM JESUS DO AMPARO       0
BOM JESUS DO GALHO        1
BRAGANÇA PAULISTA         9
BROTAS                    0
CAJOBI                    0
CAMANDUCAIA               9
CAMPINAS                  0
CAMPOS DOS GOYTACAZES     0
CASIMIRO DE ABREU         0
CONFINS                   0
COROMANDEL                0
COSMÓPOLIS                0
DIAMANTINA                0
DIVINÓPOLIS               0
GUARULHOS                 0
IPATINGA                  0
IPERÓ                     3
ITANHAÉM                  1
ITÁPOLIS                  0
JUIZ DE FORA              0
JUNDIAÍ                   7
LINHARES                  0
MACAÉ                     0
MANGARATIBA               0
MARÍLIA                   0
MO

In [58]:
# totais de recomendações por aerodromo e qeu tenham recomendacoes
filtro = dfsudeste2010.total_recomendacoes > 0
dfsudeste2010.loc[filtro].groupby(["ocorrencia_aerodromo"],dropna=False).total_recomendacoes.sum()

ocorrencia_aerodromo
SBBP     9
SBGL     3
SBJD     7
SBJR     3
SBMT    27
SBPR     1
SBRJ    19
SDOI     1
NaN     45
Name: total_recomendacoes, dtype: int64

In [60]:
# totais de recomendacoes por cidade e mes
dfsudeste2010.loc[filtro].groupby(["ocorrencia_cidade",dfsudeste2010.ocorrencia_dia.dt.month],dropna=False).total_recomendacoes.sum()

ocorrencia_cidade      ocorrencia_dia
BELO HORIZONTE         2                  1
                       3                  1
BOITUVA                7                  1
BOM JESUS DO GALHO     12                 1
BRAGANÇA PAULISTA      11                 9
CAMANDUCAIA            7                  9
IPERÓ                  1                  3
ITANHAÉM               9                  1
JUNDIAÍ                5                  5
                       6                  2
NOVO HORIZONTE         7                  3
RESENDE                4                  8
RIO DE JANEIRO         4                  3
                       7                  3
                       8                 19
SÃO BERNARDO DO CAMPO  1                  2
SÃO PAULO              1                  8
                       2                 14
                       6                  2
                       11                 6
                       12                11
TIMÓTEO                2              